In [ ]:
#Créer un programme qui permet de découper votre base de données X avec un échantillonnage stratifié 
par rapport aux labels en deux sous-ensembles d’apprentissage A et de test T de tailles respectivement 
1/2 et 1/2. 
Fichier de données à utiliser : « Wave.txt ». Cette base appelée « les vagues de Brieman » contient 
5000 individus, 40 variables et 3 classes. 

In [ ]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('Wave.txt', sep='\t', header=None)

#Créer un programme qui permet de découper votre base de données X avec un échantillonnage stratifié par rapport aux labels en deux sous-ensembles d’apprentissage A et de test T de tailles respectivement 
#1/2 et 1/2. 

def stratified_split(df):
    from sklearn.model_selection import train_test_split

   
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Effectuer un échantillonnage stratifié
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=42)

    # Combiner les ensembles d'apprentissage et de test
    A = pd.concat([X_train, y_train], axis=1)
    T = pd.concat([X_test, y_test], axis=1)

    return A, T


A, T = stratified_split(df)

